In [1]:
import re
import time
import os.path
import numpy as np
import pandas as pd

import scipy.io as sio
import scipy.sparse as sp

import tensorflow as tf
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
from sklearn.metrics import confusion_matrix, silhouette_score, davies_bouldin_score


In [2]:
def read_dataset(dataset):
    data = sio.loadmat(os.path.join('data', f'{dataset}.mat'))
    features = data['fea'].astype(float)
    adj = data['W']
    adj = adj.astype(float)

    if not sp.issparse(adj):
        adj = sp.csc_matrix(adj)
    if sp.issparse(features):
        features = features.toarray()

    labels = data['gnd'].reshape(-1) - 1
    n_classes = len(np.unique(labels))
    
    return adj, features, labels, n_classes

def aug_normalized_adjacency(adj, add_loops=True):
    if add_loops:
        adj = adj + sp.eye(adj.shape[0])
    adj = sp.coo_matrix(adj)
    row_sum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(row_sum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()


def row_normalize(mx, add_loops=True):
    if add_loops:
        mx = mx + sp.eye(mx.shape[0])
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def is_close(a, b, c):
    return np.abs(a - b) < c

def preprocess_dataset(
                       adj, 
                       features, 
                       row_norm=True, 
                       sym_norm=True, 
                       feat_norm=True, 
                       tf_idf=False
                       ):
    if sym_norm:
        adj = aug_normalized_adjacency(adj, True)
    if row_norm:
        adj = row_normalize(adj, True)

    if tf_idf:
        features = TfidfTransformer().fit_transform(features).toarray()
    if feat_norm:
        features = normalize(features)
    return adj, features


In [3]:
def ordered_confusion_matrix(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)
    w = np.max(conf_mat) - conf_mat
    row_ind, col_ind = linear_sum_assignment(w)
    conf_mat = conf_mat[row_ind, :]
    conf_mat = conf_mat[:, col_ind]
    return conf_mat


def clustering_accuracy(y_true, y_pred):
    conf_mat = ordered_confusion_matrix(y_true, y_pred)
    return np.trace(conf_mat) / np.sum(conf_mat)


def clustering_f1_score(y_true, y_pred, **kwargs):
    def cmat_to_psuedo_y_true_and_y_pred(cmat):
        y_true = []
        y_pred = []
        for true_class, row in enumerate(cmat):
            for pred_class, elm in enumerate(row):
                y_true.extend([true_class] * elm)
                y_pred.extend([pred_class] * elm)
        return y_true, y_pred

    conf_mat = ordered_confusion_matrix(y_true, y_pred)
    pseudo_y_true, pseudo_y_pred = cmat_to_psuedo_y_true_and_y_pred(conf_mat)
    return f1_score(pseudo_y_true, pseudo_y_pred, **kwargs)


def output_metrics(X, y_true, y_pred):
    return [
        clustering_accuracy(y_true, y_pred),
        nmi(y_true, y_pred),
        ari(y_true, y_pred),
        clustering_f1_score(y_true, y_pred, average='macro'),
        davies_bouldin_score(X, y_pred),
        silhouette_score(X, y_pred)
    ]


def print_metrics(metrics_means, metrics_stds, time_mean=None, time_std=None):
    if time_mean is not None: print(f'time_mean:{time_mean} ', end='')
    print(f'loss_mean:{metrics_means[6]} '
          f'acc_mean:{metrics_means[0]} '
          f'ari_mean:{metrics_means[2]} '
          f'nmi_mean:{metrics_means[1]} '
          f'db_mean:{metrics_means[4]} '
          f'sil_mean:{metrics_means[5]} '
          f'f1_mean:{metrics_means[3]} ', end=' ')

    if time_std is not None: print(f'time_std:{time_std} ', end='')
    print(f'loss_std:{metrics_stds[6]} '
          f'acc_std:{metrics_stds[0]} '
          f'ari_std:{metrics_stds[2]} '
          f'nmi_std:{metrics_stds[1]} '
          f'f1_std:{metrics_stds[3]} '
          f'db_std:{metrics_stds[4]} '
          f'sil_std:{metrics_stds[5]} ')

In [4]:
def update_rule_F(XW, G, k):
    F = tf.math.unsorted_segment_mean(XW, G, k)
    return F


def update_rule_W(X, F, G):
    _, U, V = tf.linalg.svd(tf.transpose(X) @ tf.gather(F, G), full_matrices=False)
    W = U @ tf.transpose(V)
    return W


def update_rule_G(XW, F):
    centroids_expanded = F[:, None, ...]
    distances = tf.reduce_mean(tf.math.squared_difference(XW, centroids_expanded), 2)
    G = tf.math.argmin(distances, 0, output_type=tf.dtypes.int32)
    return G


def init_G_F(XW, k):
    km = KMeans(k).fit(XW)
    G = km.labels_
    F = km.cluster_centers_
    return G, F


def init_W(X, f):
    pca = PCA(f, svd_solver='randomized').fit(X)
    W = pca.components_.T
    return W


@tf.function
def train_loop(X, F, G, W, k, max_iter, tolerance):
    losses = tf.TensorArray(tf.float64, size=0, dynamic_size=True)
    prev_loss = tf.float64.max

    for i in tf.range(max_iter):

        W = update_rule_W(X, F, G)
        XW = X @ W
        G = update_rule_G(XW, F)
        F = update_rule_F(XW, G, k)

        loss = tf.linalg.norm(X - tf.gather(F @ tf.transpose(W), G))
        if prev_loss - loss < tolerance:
            break

        losses = losses.write(i, loss)
        prev_loss = loss

    return G, F, W, losses.stack()


def optimize(X, k, f, max_iter=30, tolerance=10e-7):
    # init G and F
    W = init_W(X, f)
    G, F = init_G_F(X @ W, k)
    G, F, W, loss_history = train_loop(X, F, G, W, k, max_iter, tolerance)

    return G, F, W, loss_history

In [5]:
# Parameters
dataset = 'cora'
power = 5
runs = 20
n_clusters = 0
max_iter = 30
tolerance = 10e-7


# Read the dataset
adj, features, labels, n_classes = read_dataset(dataset)
if n_clusters == 0: n_clusters = n_classes

# Process the dataset
tf_idf = (dataset == 'cora' or dataset == 'citeseer') # normalize binary word datasets
norm_adj, features = preprocess_dataset(adj, features, tf_idf=tf_idf)


run_metrics = []
times = []

X = features

for run in range(runs):
    features = X
    t0 = time.time()
    for _ in range(power):
        features = norm_adj @ features

    G, F, W, losses = optimize(features, n_clusters, n_clusters,
                               max_iter=max_iter, tolerance=tolerance)
    time_it_took = time.time() - t0
    metrics = output_metrics(features @ W, labels, G)
    run_metrics.append(metrics + [losses[-1]])
    times.append(time_it_took)

print_metrics(np.mean(run_metrics, 0), np.std(run_metrics, 0), np.mean(times), np.std(times))

c:\Users\Legion\.conda\envs\tf26\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=11.
  warnings.warn(
c:\Users\Legion\.conda\envs\tf26\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=11.
  warnings.warn(
c:\Users\Legion\.conda\envs\tf26\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=11.
  warnings.warn(
c:\Users\Legion\.conda\envs\tf26\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is kn

time_mean:0.5649384617805481 loss_mean:24.872425293728547 acc_mean:0.7362813884785817 ari_mean:0.5034910064428256 nmi_mean:0.5777454313708891 db_mean:0.6918595316858276 sil_mean:0.46830101212875297 f1_mean:0.6974509852858335  time_std:0.6557893343271808 loss_std:0.004770342434718667 acc_std:0.004536652655133224 ari_std:0.006117919887598768 nmi_std:0.005248693315405233 f1_std:0.004308452491614769 db_std:0.014364058377548682 sil_std:0.0004983991202181373 
